In [178]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import time

In [234]:
import json
import random
import numpy as np
import scipy.stats as st

class DecisionModeler:
    
    def __init__(self):
        pass
    
    def p_cc2a(self, dp, k1, k2):
        a_given_a = st.norm.cdf(k1 + dp/2)
        b_given_a = 1 - st.norm.cdf(k2 + dp/2)
        a_given_b = st.norm.cdf(k1 - dp/2)
        b_given_b = 1 - st.norm.cdf(k2 - dp/2)
        
        pAA = a_given_a*b_given_a + b_given_a*a_given_a
        pBB = a_given_b*b_given_b + b_given_b*a_given_b
        pAB = a_given_a*b_given_b + b_given_a*a_given_b
        pBA = a_given_b*b_given_a + b_given_b*a_given_a

        return pAA, pBB, pAB, pBA
    
    def generate_2_param_model(self, dp_vals, k_vals, p_function, filepath):
        model = dict()
        
        counter = progress = 0
        total = len(dp_vals)
        
        for dp in dp_vals:
            dp_list = []
            for k1 in k_vals:
                for k2 in [k for k in k_vals if k >= k1]:
                    dp_list.append(p_function(dp, k1, k2))
            model[dp] = dp_list
            
            counter += 1
            if counter/total >= progress+0.1:
                progress += 0.1
                print(f'{round(progress*100, 2)}% evaluated.')
            
        with open(filepath, 'w') as file:
            json.dump(model, file)
            print('Successfully written data to file.')
    
    def get_distance(self, pHuman, pModel):
        return sum([(pH-pM)**2 for pH, pM in zip(pHuman, pModel)])
    
    def get_chi_squared(self, pHuman, pModel):
        return sum([(pH-pM)**2/pH for pH, pM in zip(pHuman, pModel)])
    
    def search_dp(self, pHuman_vals, dp_vals=None, k_vals=None, model_input=None):
        '''
        Routine:
            Scan dp values, (k1,k2) vals
            Minimize euclidean distance
            Keep track of chi-squared and best dp
        '''
        
        min_dist = np.inf
        best_dp = 0
        dist_arr = np.empty(len(pHuman_vals))
        chi_squared_arr = np.empty(len(pHuman_vals))
        
        if model_input != None:
            with open(model_input, 'r') as file:
                model = json.load(file)
                
            for dp in model.keys():
                dist_arr.fill(np.inf)
                chi_squared_arr.fill(np.inf)
                
                for pModel in model[dp]:
                    for index, pHuman in enumerate(pHuman_vals):
                        dist = self.get_distance(pHuman, pModel)
                        chi_squared = self.get_chi_squared(pHuman, pModel)
                        dist_arr[index] = min(dist_arr[index], dist)
                        chi_squared_arr[index] = min(chi_squared_arr[index], chi_squared)
                if sum(dist_arr) < min_dist:
                    min_dist = sum(dist_arr)
                    best_dp = float(dp)
        else:
            for dp in dp_vals:
                dist_arr.fill(np.inf)
                chi_squared_arr.fill(np.inf)
                for k1 in k_vals:
                    for k2 in k_vals:
                        pModel = p_cc2a(dp, k1, k2)
                        for index, pHuman in enumerate(pHuman_vals):
                            dist = self.get_distance(pHuman, pModel)
                            chi_squared = self.get_chi_squared(pHuman, pModel)
                            dist_arr[index] = min(dist_arr[index], dist)
                            chi_squared_arr[index] = min(chi_squared_arr[index], chi_squared)
                if sum(dist_arr) < min_dist:
                    min_dist = sum(dist_arr)
                    best_dp = dp
                
        return best_dp, sum(chi_squared_arr)
                                       

In [235]:
modeler = DecisionModeler()

dp_vals = np.linspace(0, 2.5, 11)
k_vals = np.linspace(-4, 4, 101)
modeler.generate_2_param_model(dp_vals, k_vals, modeler.p_cc2a, 'cc2a_exp.json')

10.0% evaluated.
20.0% evaluated.
30.0% evaluated.
40.0% evaluated.
50.0% evaluated.
60.0% evaluated.
70.0% evaluated.
80.0% evaluated.
90.0% evaluated.
100.0% evaluated.
Successfully written data to file.


In [229]:
modeler = DecisionModeler()

pModel = (0.1, 0.5, 0.29, 0.29)
modeler.search_dp([pModel], model_input='cc2a_exp.json')

(1.25, 0.10807545276670308)

In [208]:
with open('cc2a.json') as file:
    data = json.load(file)

for dp in data.keys():
    [print(tup) for tup in data[dp]]

[6.334047753112123e-05, 6.334047753112123e-05, 6.334047753112123e-05, 6.334047753112123e-05]
[6.333790096144912e-05, 6.333790096144912e-05, 6.333790096144912e-05, 6.333790096144912e-05]
[6.333240533296651e-05, 6.333240533296651e-05, 6.333240533296651e-05, 6.333240533296651e-05]
[6.332114172973191e-05, 6.332114172973191e-05, 6.332114172973191e-05, 6.332114172973191e-05]
[6.329895864263084e-05, 6.329895864263084e-05, 6.329895864263084e-05, 6.329895864263084e-05]
[6.32569777722201e-05, 6.32569777722201e-05, 6.32569777722201e-05, 6.32569777722201e-05]
[6.318063536501948e-05, 6.318063536501948e-05, 6.318063536501948e-05, 6.318063536501948e-05]
[6.304723243998205e-05, 6.304723243998205e-05, 6.304723243998205e-05, 6.304723243998205e-05]
[6.282323138083258e-05, 6.282323138083258e-05, 6.282323138083258e-05, 6.282323138083258e-05]
[6.246180476921153e-05, 6.246180476921153e-05, 6.246180476921153e-05, 6.246180476921153e-05]
[6.190143380490739e-05, 6.190143380490739e-05, 6.190143380490739e-05, 6.19

[0.0013803950163009374, 0.0007478441974742237, 0.0017663856149028737, 0.0017663856149028737]
[0.0009287482056160694, 0.0005468048607883998, 0.0012821561527377396, 0.0012821561527377396]
[0.0006036923519130886, 0.0003869501154679041, 0.000901091923320445, 0.000901091923320445]
[0.0003788890261080583, 0.000264810684574757, 0.0006126774143389207, 0.0006126774143389207]
[0.00022949505026318147, 0.00017513554564125476, 0.0004027457354851277, 0.0004027457354851277]
[0.00013409492212615402, 0.00011186902364065525, 0.0002558014888845877, 0.0002558014888845877]
[7.55552550995152e-05, 6.897848321535602e-05, 0.00015689735005081418, 0.00015689735005081418]
[4.103795297555568e-05, 4.103795297555572e-05, 9.288866828013544e-05, 9.288866828013544e-05]
[2.1480665187521338e-05, 2.3547814892253516e-05, 5.305935806755055e-05, 5.305935806755055e-05]
[1.0832728318299749e-05, 1.302729515319368e-05, 2.9231514552181013e-05, 2.9231514552181013e-05]
[5.262060905737779e-06, 6.9464156120421835e-06, 1.5526961812247

[0.08050300873546813, 0.006859587189069177, 0.047218906891614204, 0.047218906891614204]
[0.07703888394367753, 0.006810076649885757, 0.046765949713025695, 0.046765949713025695]
[0.07272547619413848, 0.006734828484890272, 0.04611453815178857, 0.04611453815178857]
[0.06756448125294919, 0.006624933022272824, 0.04520747444715515, 0.04520747444715515]
[0.06163068913367503, 0.006470709972774137, 0.04398545115741625, 0.04398545115741625]
[0.05507501694728022, 0.006262738117721423, 0.04239378708106276, 0.04239378708106276]
[0.048115354756388486, 0.005993246247555675, 0.04039101094183032, 0.04039101094183032]
[0.04101557076069751, 0.005657684340558137, 0.037958142930118344, 0.037958142930118344]
[0.034055908569805775, 0.005256184318990457, 0.035106931716186555, 0.035106931716186555]
[0.027500236383410977, 0.0047945644780460475, 0.03188503613085516, 0.03188503613085516]
[0.021566444264136828, 0.0042845654266573975, 0.028376390117185544, 0.028376390117185544]
[0.01640544932294752, 0.00374313793998

[0.008136389407229085, 0.008136389407229098, 0.06673354975269764, 0.06673354975269764]
[0.004840651030876013, 0.006237284002628668, 0.05111461775080718, 0.05111461775080718]
[0.0027775935729776966, 0.00463416906664982, 0.03795201701935834, 0.03795201701935834]
[0.0015366407225622378, 0.0033338012997464308, 0.02728839678195961, 0.02728839678195961]
[0.0008193677183010098, 0.002320231579757198, 0.01898430218227431, 0.01898430218227431]
[0.00042098653782040554, 0.0015610827993915674, 0.01276890571461744, 0.01276890571461744]
[0.00020836946550281508, 0.0010147139353778125, 0.008297874382470148, 0.008297874382470148]
[9.933035411657424e-05, 0.0006368541419072417, 0.005206944762736851, 0.005206944762736851]
[4.5596106234086024e-05, 0.000385745860228833, 0.0031534277439683963, 0.0031534277439683963]
[2.0150981220308115e-05, 0.00022539292689995893, 0.0018423624877338157, 0.0018423624877338157]
[8.572745649309228e-06, 0.00012699675587664406, 0.0010379867865615681, 0.0010379867865615681]
[3.5102

[0.02275013194817921, 0.022750131948179195, 0.2505175685036596, 0.2505175685036596]
[0.01390344751349859, 0.019143794252351316, 0.21068645054591512, 0.21068645054591512]
[0.008197535924596155, 0.015678401689677813, 0.17247562421877236, 0.17247562421877236]
[0.004661188023718732, 0.012478589232027392, 0.1372326015176116, 0.1372326015176116]
[0.0025551303304279793, 0.009639476543412729, 0.10598582884386024, 0.10598582884386024]
[0.0013498980316301035, 0.007218855922425048, 0.0793583373240647, 0.0793583373240647]
[0.0006871379379158604, 0.005235700361554656, 0.057550467589608294, 0.057550467589608294]
[0.0003369292656768552, 0.003674449306524866, 0.04038599480213683, 0.04038599480213683]
[0.00015910859015755285, 0.0024933822336634056, 0.027403265591199114, 0.027403265591199114]
[7.234804392508565e-05, 0.001634839001518492, 0.017966805484008374, 0.017966805484008374]
[3.167124183311998e-05, 0.001035137007319128, 0.01137578649902027, 0.01137578649902027]
[1.3345749015902797e-05, 0.000632610

[0.004340695091932377, 0.34868702255906575, 0.3616246941673133, 0.3616246941673133]
[0.002272067956373353, 0.2795489338626778, 0.2896292775495447, 0.2896292775495447]
[0.0011458069234469143, 0.2175899948234547, 0.22528560342231588, 0.22528560342231588]
[0.0005565823899791126, 0.16423481652862393, 0.16996886269833547, 0.16996886269833547]
[0.00026036717594782235, 0.12008436555615305, 0.12424151617050969, 0.12424151617050969]
[0.00011727413157066944, 0.0849785041245727, 0.08790414749802743, 0.08790414749802743]
[5.085179508967354e-05, 0.05815534083548673, 0.060150406340575883, 0.060150406340575883]
[2.122435931651064e-05, 0.03846169336230401, 0.03977815303457164, 0.03977815303457164]
[8.52569341741429e-06, 0.02456766882047323, 0.025407350005076233, 0.025407350005076233]
[3.295640797757286e-06, 0.015148434090360421, 0.015665707673272185, 0.015665707673272185]
[1.2257961245472266e-06, 0.009012386136596315, 0.009319954419089647, 0.009319954419089647]
[4.386542988612452e-07, 0.00517135931727